In [1]:
!nvidia-smi

Wed Nov  1 15:53:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:52:00.0 Off |                  Off |
| 30%   27C    P8    21W / 300W |      1MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import os.path as osp
import cv2
import numpy as np
import matplotlib.pyplot as plt
from rasterio import features
from PIL import Image

In [9]:
def create_label(mask_path, save_path):
    arr = np.asarray(Image.open(mask_path))

    # There may be a better way to do it, but this is what I have found so far
    cords = list(features.shapes(arr, mask=(arr >0)))[0][0]['coordinates'][0]
    label_line = '0 ' + ' '.join([f'{int(cord[0])/arr.shape[0]} {int(cord[1])/arr.shape[1]}' for cord in cords])

    with open(save_path, 'w') as f:
        f.write(label_line)
    return label_line

def convert_data(mask_dir, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    mask_names = os.listdir(mask_dir)
    
    for mask_name in mask_names:
        mask_path = osp.join(mask_dir, mask_name)
        mask_txt_name = mask_name.replace('.png', '.txt')
        out_path = osp.join(save_dir, mask_txt_name)
        create_label(mask_path, out_path)

In [10]:
convert_data(
    '/home/mpham/workspace/ucc-ai-quest-2023/data/warmup/ann/train', 
    '/home/mpham/workspace/ucc-ai-quest-2023/data/yolov8/train/labels'
)

convert_data(
    '/home/mpham/workspace/ucc-ai-quest-2023/data/warmup/ann/valid', 
    '/home/mpham/workspace/ucc-ai-quest-2023/data/yolov8/valid/labels'
)

In [3]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")

results = model.train(
        batch=8,
        device=0,
        data="/home/mpham/workspace/ucc-ai-quest-2023/infection/data.yml",
        epochs=7,
        imgsz=480,
    )

Ultralytics YOLOv8.0.203 🚀 Python-3.10.13 torch-2.1.0 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/home/mpham/workspace/ucc-ai-quest-2023/infection/data.yml, epochs=7, patience=50, batch=8, imgsz=480, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, for